In [4]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from src.data import make_dataset
from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_A_ANLIZAR = 112103  # Este es el Torax

## 1. Obtencion de Métricas de Egresos Hospitalarios

En este apartado se quieren calculcar diversas métricas relevantes para el análisis de los
egresos hospitalarios en Chile. Las métricas a calcular son las siguientes:

1. Cantidad de Egresos Hospitalarios
2. Dias de Estada Totales
3. Cantidad de Intervenciones Quirúrgicas
4. Cantidad de Muertos

Todas estas métricas serán calculadas para cada Año, Establecimiento de Salud y Diagnóstico
distinto que exista en la base de datos. Por ejemplo:

"En el año 2019, el Hospital San José (código XXXXXX) tuvo XX egresos, con XX dias estada promedio,
con XX intervenciones quirúrgicas y XX muertos para el diagnóstico A24.4"s


In [6]:
df_nacional = pl.scan_csv(
    "../data/processed/egresos_procesados.csv",
    infer_schema_length=10000,
    null_values="Extranjero",
)

In [7]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = build_features.obtener_metricas_egresos(df_nacional, AGRUPACION).collect()

In [8]:
display(metricas.sample(10))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos
i64,i64,str,str,u32,i64,i64,i64
2007,121110,"""Hospital Dr. A…","""S925""",1,13,0,0
2003,103102,"""Hospital Dr. M…","""T188""",1,1,0,0
2018,121120,"""Hospital de Lo…","""O260""",5,6,0,0
2005,117101,"""Hospital Clíni…","""K219""",4,37,0,0
2013,112212,"""Clínica Las Co…","""M419""",6,33,6,0
2002,116110,"""Hospital San J…","""B378""",1,14,0,0
2014,104103,"""Hospital Provi…","""N039""",2,59,0,0
2019,112244,"""Hospital del T…","""N350""",1,15,1,0
2018,121209,"""Hospital Clíni…","""N189""",8,35,2,0


- Al ver una muestra del calculo realizado, se puede observar la cantidad de egresos, dias de
  estada, numero de intervenciones quirurgicas y cantidad de muertos para 10 duplas de
  hospitales - diagnosticos distintos.


Si bien se tienen las metricas calculadas, ahora se quiere responder a las siguientes preguntas:

- En qué diagnósticos se destaca cada hospital productivamente? Esto significa, en qué diagnósticos
  cada hospital tiene una mayor cantidad de egresos que otros hospitales?

- El hospital tiene un desempeño similar dentro de distintos grupos/estratos de hospitales?
  El hospital se desempeñara de la misma forma nacionalmente, dentro del estrato público, privado,
  etc...?

Para responder a la pregunta se realizará un ranking de todos los hospitales para cada
diagnóstico y cada año observado. Este ranking se realizará para distintos grupos/estratos de
hospitales (Nacional, Hospitales Públicos, Hospitales Privados, Hospitales GRD)


## 2. Análisis de Estratos

En este análisis se quiere observar el ranking de cada hospital/clínica del país para cada 
año y diagnóstico observado. Además, el ranking se realizará en los siguientes estratos:

1. Nacional
2. Públicos
3. Privados
4. Adscritos a GRD
5. INT

In [9]:
dict_estratos = build_features.obtener_diccionario_estratos(df_nacional, HOSPITAL_A_ANLIZAR)

In [10]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales = build_features.agregar_ranking_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

rankings_hospital_analizado = rankings_nacionales.filter(
    pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANLIZAR
)

In [11]:
display(rankings_nacionales.head(8))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,str,str,str,str,u32,i64,i64,i64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64
2020,104103,"""Hospital Provi…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",6,32,2,0,1,15,0.4,1,14,0.428571,null,null,null,1,6,1.0,null,null,null
2020,125102,"""Hospital Dr. L…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",3,8,0,0,2,15,0.2,2,14,0.214286,null,null,null,null,null,null,null,null,null
2020,125103,"""Hospital Lord …","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",2,349,0,0,3,15,0.133333,3,14,0.142857,null,null,null,null,null,null,null,null,null
2020,107107,"""Hospital de Pe…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",1,140,0,0,4,15,0.066667,4,14,0.071429,null,null,null,null,null,null,null,null,null
2020,116101,"""Hospital de Te…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",1,1,0,0,4,15,0.066667,4,14,0.071429,null,null,null,null,null,null,null,null,null
2020,118106,"""Hospital de Lo…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",1,1,0,0,4,15,0.066667,4,14,0.071429,null,null,null,null,null,null,null,null,null
2020,102200,"""Clínica Iquiqu…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",1,3,1,0,4,15,0.066667,null,null,null,1,1,1.0,null,null,null,null,null,null
2020,112211,"""Clínica Indisa…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Persona en con…","""Z768""",58,118,14,1,1,299,0.19398,null,null,null,1,149,0.389262,null,null,null,null,null,null


Por lo tanto, se pueden observar todas las métricas de egresos hospitalarios nacionales para
cada estrato definido. Además, se observa que en casos de empate en cantidad de egresos,
el ranking es repetido.

## 3. Diagnósticos más relevantes del Instituto Nacional del Tórax

En este apartado se quieren responder las siguientes preguntas:

1. ¿Cuáles son los **diagnósticos más relevantes productivamente** en el Instituto Nacional del
Tórax?
2. ¿En qué diagnósticos el INT tiene una mayor cantidad de Egresos que cualquier otro establecimiento
de salud Nacional? En otras palabras, ¿En qué diagnósticos el **INT tiene un Ranking 1 Nacional**?

Estas preguntas se quieren responder tanto para diagnósticos ambulatorios (n_int_q = 0) y 
quirúrgicos (n_int_q > 0).

Además, se sabe que desde el 2011 el registro de Egresos Hospitalarios DEIS fue llevado por la
unidad de Grupo Relacionado al Diagnóstico (GRD). Debido a lo anterior, la calidad de la codificación
de diagnósticos es mucho mayor. Tomado de lo anterior, **se obtendrán los diagnósticos que tengan
una alta productividad desde el 2012 hasta 2020**.

In [12]:
DIAGS_MAS_RELEVANTES_QUIRURGICOS = rankings_hospital_analizado.filter(
    (pl.col("ranking_nacionales_n_egresos") == 1)
    & (pl.col("n_int_q") > 0)
    & (pl.col("ANO_EGRESO") > 2011)
).sort(by=["ANO_EGRESO", "n_egresos", "n_int_q"], descending=True)


# Obtiene los 20 diagnosticos con la mayor cantidad de egresos e int_q
veinte_mas_relevantes_por_anio_quir = DIAGS_MAS_RELEVANTES_QUIRURGICOS.group_by(
    ["ANO_EGRESO"]
).head(20)

In [13]:
DIAGS_MAS_RELEVANTES_QUIRURGICOS = veinte_mas_relevantes_por_anio_quir.select(
    pl.col("DIAG1").unique()
).sort(by="DIAG1")

In [14]:
print(list(DIAGS_MAS_RELEVANTES_QUIRURGICOS.to_series()))
print(f"Hay {len(DIAGS_MAS_RELEVANTES_QUIRURGICOS)} diagnosticos quirurgicos altamente "
      f"relevantes para el país")

['C33X', 'C340', 'C341', 'C342', 'C343', 'C381', 'C384', 'C450', 'C780', 'C782', 'D143', 'D381', 'E848', 'I080', 'I081', 'I340', 'I350', 'I351', 'I352', 'I420', 'I456', 'I472', 'I495', 'I710', 'I712', 'J398', 'J47X', 'J679', 'J841', 'J848', 'J849', 'J860', 'J869', 'J90X', 'J91X', 'J931', 'J955', 'J980', 'M348', 'Q211', 'Q231', 'Q676', 'T820', 'T821', 'Z450']
Hay 45 diagnosticos quirurgicos altamente relevantes para el país


In [15]:
DIAGS_MAS_RELEVANTES_AMBULATORIOS = rankings_hospital_analizado.filter(
    (pl.col("ranking_nacionales_n_egresos") == 1)
    & (pl.col("n_int_q") == 0)
    & (pl.col("ANO_EGRESO") > 2011)
).sort(by=["ANO_EGRESO", "n_egresos", "n_int_q"], descending=True)

veinte_mas_relevantes_por_anio_ambul = DIAGS_MAS_RELEVANTES_AMBULATORIOS.group_by(
    ["ANO_EGRESO"]
).head(20)

DIAGS_MAS_RELEVANTES_AMBULATORIOS = veinte_mas_relevantes_por_anio_ambul.select(
    pl.col("DIAG1").unique()
).sort(by="DIAG1")

print(list(DIAGS_MAS_RELEVANTES_AMBULATORIOS.to_series()))
print(f"Hay {len(DIAGS_MAS_RELEVANTES_AMBULATORIOS)} diagnosticos ambulatorios altamente "
      f"relevantes para el país")

['A022', 'A151', 'A152', 'A154', 'A155', 'A157', 'A190', 'B371', 'B441', 'B460', 'B59X', 'B948', 'C33X', 'C382', 'C783', 'D611', 'D860', 'D862', 'E662', 'E854', 'G580', 'I062', 'I082', 'I150', 'I232', 'I253', 'I270', 'I279', 'I280', 'I421', 'I422', 'I424', 'I427', 'I445', 'I447', 'I451', 'I456', 'I458', 'I471', 'I472', 'I483', 'I493', 'I494', 'I513', 'I715', 'I716', 'J150', 'J155', 'J430', 'J441', 'J632', 'J677', 'J678', 'J679', 'J684', 'J700', 'J708', 'J82X', 'J840', 'J848', 'J940', 'J951', 'J958', 'L581', 'M051', 'M348', 'M350', 'M358', 'M485', 'M941', 'Q218', 'Q222', 'Q225', 'Q228', 'Q233', 'Q240', 'Q241', 'Q257', 'Q258', 'Q874', 'R048', 'S203', 'T366', 'T371', 'T380', 'T822', 'T862', 'T863', 'T868', 'Z030', 'Z100', 'Z118', 'Z136', 'Z291', 'Z292', 'Z488', 'Z514', 'Z516', 'Z541', 'Z748', 'Z851', 'Z858', 'Z877', 'Z911', 'Z924', 'Z952', 'Z953', 'Z998']
Hay 108 diagnosticos ambulatorios altamente relevantes para el país


Por lo tanto, se reportan los diagnósticos altamente relevantes para el país productivamente
hablando. En el ámbito quirúrgico, existen 45 diagnósticos dónde el INT tiene la mayor cantidad
de Egresos Hospitalarios del país entre 2012 y 2020. En el ámbito "ambulatorio" (egresos sin int_q),
existen 108 diagnósticos con una alta producción a nivel país.

Para el MINSAL, se quiere reportar los diagnósticos quirúrgicos que hayan tenido un ranking 1 o 2 
en el año 2019 para el INT.

In [16]:
DIAGS_1_O_2_EN_2019 = rankings_hospital_analizado.filter(
    (pl.col("ANO_EGRESO") == 2019)
    & (
        (pl.col("ranking_nacionales_n_egresos") == 1)
        | (pl.col("ranking_nacionales_n_egresos") == 2)
    )
    & (pl.col("DIAG1").is_in(list(DIAGS_MAS_RELEVANTES_QUIRURGICOS.to_series())))
).select(pl.col("DIAG1").unique()).sort("DIAG1")

print(list(DIAGS_1_O_2_EN_2019.to_series()))
print(
    f"Hay {len(DIAGS_1_O_2_EN_2019)} diagnosticos relevantes productivamente Y que "
    f"tenian ranking 1 o 2 en 2019"
)

['C33X', 'C340', 'C341', 'C342', 'C343', 'C381', 'C384', 'C450', 'C780', 'C782', 'D143', 'D381', 'E848', 'I080', 'I081', 'I340', 'I351', 'I352', 'I420', 'I456', 'I472', 'I495', 'I710', 'I712', 'J47X', 'J679', 'J841', 'J848', 'J849', 'J860', 'J869', 'J955', 'M348', 'T820', 'T821', 'Z450']
Hay 36 diagnosticos relevantes productivamente Y que tenian ranking 1 o 2 en 2019


In [22]:
tabla_resumen_egresos_torax_relevantes = (
    rankings_hospital_analizado.filter(
        ((pl.col("ANO_EGRESO") >= 2016) & (pl.col("ANO_EGRESO") <= 2019))
        & (pl.col("DIAG1").is_in(list(DIAGS_1_O_2_EN_2019.to_series())))
    )
    .with_columns((pl.col("DIAG1") + " - " + pl.col("Descripción")).alias("Diagnóstico"))
    .to_pandas()
    .pivot(
        index=["Diagnóstico"],
        columns="ANO_EGRESO",
        values=[
            "ranking_nacionales_n_egresos",
            "n_egresos",
            "total_nacionales_n_egresos",
            "%_nacionales_n_egresos",
        ],
    )
).sort_values(
    by=[("ranking_nacionales_n_egresos", 2019), ("n_egresos", 2019)], ascending=[True, False]
)

In [23]:
resumen_egresos_torax_y_nacional_relevantes = (
    tabla_resumen_egresos_torax_relevantes[("n_egresos")].astype(int).astype(str)
    + " ("
    + tabla_resumen_egresos_torax_relevantes[("total_nacionales_n_egresos")].astype(int).astype(str)
    + "; "
    + round((tabla_resumen_egresos_torax_relevantes[("%_nacionales_n_egresos")] * 100), 1).astype(
        str
    )
    + "%)"
)

posiciones_torax_egresos_relevantes = tabla_resumen_egresos_torax_relevantes[
    ("ranking_nacionales_n_egresos")
].astype(int)

resumen_egresos_y_posiciones_relevantes = pd.concat(
    [resumen_egresos_torax_y_nacional_relevantes, posiciones_torax_egresos_relevantes], axis=1
)

resumen_egresos_y_posiciones_relevantes.columns = [
    "2016",
    "2017",
    "2018",
    "2019",
    "2016",
    "2017",
    "2018",
    "2019",
]

display(resumen_egresos_y_posiciones_relevantes)

,2016,2017,2018,2019,2016,2017,2018,2019
Diagnóstico,,,,,,,,
"C341 - Tumor maligno del lóbulo superior, bronquio o pulmón",180 (400; 45.0%),180 (372; 48.4%),220 (478; 46.0%),220 (493; 44.6%),1,1,1,1
Z450 - Asistencia y ajuste de marcapaso cardíaco,101 (1146; 8.8%),71 (718; 9.9%),132 (649; 20.3%),137 (685; 20.0%),2,1,1,1
J841 - Otras enfermedades pulmonares intersticiales con fibrosis,124 (1617; 7.7%),108 (1666; 6.5%),108 (1633; 6.6%),122 (1697; 7.2%),1,1,1,1
"C343 - Tumor maligno del lóbulo inferior, bronquio o pulmón",120 (241; 49.8%),118 (258; 45.7%),124 (293; 42.3%),120 (279; 43.0%),1,1,1,1
"D381 - Tumor de comportamiento incierto o desconocido de la tráquea, de los bronquios y del pulmón",91 (637; 14.3%),36 (553; 6.5%),85 (641; 13.3%),97 (654; 14.8%),1,1,1,1
J679 - Neumonitis debida a hipersensibilidad a polvo orgánico no especificado,53 (68; 77.9%),77 (106; 72.6%),70 (101; 69.3%),69 (109; 63.3%),1,1,1,1
C780 - Tumor maligno secundario del pulmón,43 (328; 13.1%),57 (373; 15.3%),63 (379; 16.6%),62 (430; 14.4%),1,1,1,1
J848 - Otras enfermedades pulmonares intersticiales especificadas,40 (98; 40.8%),25 (113; 22.1%),53 (172; 30.8%),52 (158; 32.9%),1,1,1,1
I351 - Insuficiencia (de la válvula) aórtica,19 (209; 9.1%),37 (318; 11.6%),52 (327; 15.9%),49 (313; 15.7%),1,2,1,1


In [24]:
resumen_egresos_y_posiciones_relevantes = resumen_egresos_y_posiciones_relevantes.reset_index()

In [25]:
visualize.add_dataframes_to_powerpoint(
    [resumen_egresos_y_posiciones_relevantes],
    "../reports/presentacion.pptx",
    font_size=8,
    cell_width=33,
    cell_height=0.5,
    font_family="Open Sans",
    max_cell_characters=100,
)

Para hacer un contorl de lo que se calculo, se observará una muestra de 10 hospitales - diagnósticos
distintos


In [26]:
display(rankings_nacionales.sample(10))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,str,str,str,str,u32,i64,i64,i64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64
2012,113190,"""Hospital Parro…","""Cap.01 CIERTA…","""B20-B23 ENFER…","""B20 ENFERMEDAD…","""Enfermedad por…","""B206""",1,7,0,0,36,214,0.004673,28,170,0.005882,null,null,null,null,null,null,null,null,null
2020,123100,"""Hospital Base …","""Cap.19 LESION…","""T66-T78 OTROS…","""T75 EFECTOS DE…","""Ahogamiento y …","""T751""",2,24,0,0,14,106,0.018868,12,80,0.025,null,null,null,12,75,0.026667,null,null,null
2007,101214,"""Clínica Arica""","""Cap.18 SÍNTOM…","""R30-R39 SÍNTO…","""R31 HEMATURIA,…","""Hematuria, no …","""R31X""",9,17,2,0,41,1380,0.006522,null,null,null,13,356,0.025281,null,null,null,null,null,null
2009,105101,"""Hospital San P…","""Cap.18 SÍNTOM…","""R50-R69 SÍNTO…","""R69 CAUSAS DE …","""Causas de morb…","""R69X""",1,8,0,0,20,630,0.001587,12,423,0.002364,null,null,null,8,302,0.003311,null,null,null
2009,108100,"""Hospital de Sa…","""Cap.08 ENFERM…","""H65-H75 ENFER…","""H73 OTROS TRAS…","""Otros trastorn…","""H738""",1,1,1,0,17,119,0.008403,9,90,0.011111,null,null,null,9,90,0.011111,null,null,null
2014,120101,"""Complejo Asist…","""Cap.07 ENFERM…","""H10-H13 TRAST…","""H11 OTROS TRAS…","""Hemorragia con…","""H113""",2,4,0,0,1,13,0.153846,1,11,0.181818,null,null,null,1,7,0.285714,null,null,null
2009,115103,"""Hospital Del S…","""Cap.10 ENFERM…","""J20-J22 OTRAS…","""J21 BRONQUIOLI…","""Bronquiolitis …","""J219""",2,11,0,0,108,3051,0.000656,83,2685,0.000745,null,null,null,null,null,null,null,null,null
2008,112102,"""Hospital de Ni…","""Cap.02 NEOPLA…","""C40-C41 NEOPL…","""C41 TUMOR MALI…","""Tumor maligno …","""C411""",1,5,1,0,5,25,0.04,4,18,0.055556,null,null,null,3,13,0.076923,null,null,null
2005,112249,"""Clínica Santa …","""Cap.11 ENFERM…","""K55-K64 OTRAS…","""K61 ABSCESO DE…","""Absceso anorre…","""K612""",1,2,0,0,53,336,0.002976,null,null,null,26,154,0.006494,null,null,null,null,null,null


En la muestra anterior se puede observar las métricas previamente calculada, y el ranking
respectivo que tiene cada hospital para diagnósticos en cada año específico.


In [27]:
rankings_nacionales.to_pandas().to_csv(
    "../data/interim/ranking_nacional_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

rankings_hospital_analizado.to_pandas().to_csv(
    "../data/interim/ranking_torax_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

## Metricas para personas sobre 14 años

En este análisis se quiere ver el ranking de egresos pero solamente tomando en cuenta a personas
mayores a 14 años. Esto, para dilucidar si el hospital del tórax es altamente relevante para tratar
patologías congénitas de adultos.


In [28]:
df_nacional_mayores_a_14 = df_nacional.filter(pl.col("EDAD_A_OS") > 14)

In [29]:
metricas_sobre_14 = build_features.obtener_metricas_egresos(
    df_nacional_mayores_a_14, AGRUPACION
).collect()

In [30]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales_sobre_14 = build_features.agregar_ranking_estratos(
    metricas_sobre_14, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

diags_congenitos_relevantes = rankings_nacionales_sobre_14.filter(
    (pl.col("DIAG1").is_in(["Q230", "Q211", "Q231"])) & (pl.col("ANO_EGRESO") == 2019)
)

In [31]:
diags_congenitos_relevantes.to_pandas().to_csv(
    "../data/interim/diags_congenitas_sobre_14_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

## 3. Análisis de procedimientos DEIS

En este análisis se quieren responder las siguientes preguntas:

1. ¿Cuál es el procedimiento más frecuente dentro de los pacientes que se les realizó un procedimiento
   quirúrgico?


In [32]:
proced = (
    df_nacional.filter(pl.col("INTERV_Q") == 1)
    .groupby(["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "GLOSA_PROCED_PPAL"])
    .agg(pl.count())
).collect()

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_11568/3397463805.py:3: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby(["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "GLOSA_PROCED_PPAL"])


In [33]:
display(proced)

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_PROCED_PPAL,count
i64,i64,str,u32
2001,124250,null,1122
2001,112212,null,7433
2001,102200,null,2700
2001,120106,null,24
2001,112240,null,4644
2001,121119,null,371
2001,105203,null,701
2001,111230,null,2400
2001,105102,null,4171


Los resultados muestran el desglose por año y recinto hospitalario junto a la cantidad de
procedimientos realizados en cada intervención quirúrgica. Sin embargo, se puede observar que
existe una cantidad importante de casos donde la glosa del procedimiento fue nula.

Ahora, se quiere calcular cuántas intervenciones quirúrgicas carecen de una glosa de procedimiento.


In [34]:
display(proced.groupby("GLOSA_PROCED_PPAL").agg(pl.col("count").sum()).sort("count"))

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_11568/989951252.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  display(proced.groupby("GLOSA_PROCED_PPAL").agg(pl.col("count").sum()).sort("count"))


GLOSA_PROCED_PPAL,count
str,u32
"""Tratamiento po…",1
"""Infiltraciones…",1
"""Sondeo gastric…",1
"""Volúmenes pulm…",1
"""Braguero (Hern…",1
"""Ecografía tran…",1
"""Prótesis metál…",1
"""Aplicación de …",1
"""Punción evacua…",1


Por lo tanto, 12382642 de todas las intervenciones quirúrgicas carecen de una glosa de procedimiento.


In [35]:
proced.write_excel("../data/interim/conteo_procedimientos.xlsx")